Import libraries

In [1]:
import numpy as np
import pandas as pd
import json

from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize


import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
import folium

Get BeautifulSoup and extract Toronto table elements

In [2]:
from bs4 import BeautifulSoup
from tabulate import tabulate
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'


In [3]:
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')
table = soup.find('table')
table_rows = table.find_all('tr')

rows = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        rows.append(row)


df = pd.DataFrame(rows, columns=["Post Code", "Borough", "Neighborhood"])
df.head(10)

,Post Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


###
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [4]:
df = df.drop(df[df['Borough']=='Not assigned'].index)
df.head()

,Post Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


###
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [5]:
df.loc[df['Neighborhood']=='Not assigned', 'Neighborhood'] = df['Borough']
df[df['Borough']=="Queen's Park"]

,Post Code,Borough,Neighborhood
8,M7A,Queen's Park,Queen's Park


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma

In [6]:
def squish(group):
     return pd.Series(dict(Borough = group['Borough'].max(), 
                        Neighborhood =  ', '.join(group['Neighborhood'])))

squished = df.groupby('Post Code').apply(squish).reset_index()
print(squished.shape)
squished.head()

(103, 3)


,Post Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
geocodes = pd.read_csv('http://cocl.us/Geospatial_data')
geocodes.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
squished = squished.merge(geocodes, left_on = 'Post Code', right_on='Postal Code')
squished.head()

,Post Code,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [9]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(squished['Borough'].unique()),
        squished.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [10]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

C:\Users\Tom\Anaconda37\lib\site-packages\geopy\geocoders\osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [11]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(squished['Latitude'], squished['Longitude'], squished['Borough'], squished['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [12]:
with open(r'C:\Users\Tom\Documents\foursquare_cred.txt') as file:
    lines = file.readlines()
    CLIENT_ID, CLIENT_SECRET = lines[0], lines[1]

VERSION = '20180605' # Foursquare API version

In [13]:
neighborhood_latitude = squished.loc[3, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = squished.loc[3, 'Longitude'] # neighborhood longitude value

neighborhood_name = squished.loc[3, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Woburn are 43.7709921, -79.21691740000001.


In [14]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 2000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [15]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5bb824ea6a6071462af384e0'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4bf470256a31d13a9ec7952e-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/burger_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16c941735',
         'name': 'Burger Joint',
         'pluralName': 'Burger Joints',
         'primary': True,
         'shortName': 'Burgers'}],
       'id': '4bf470256a31d13a9ec7952e',
       'location': {'address': '1021 Markham Road',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Brimorton',
        'distance': 1144,
        'formattedAddress': ['1021 Markham Road (Brimorton)',
         'Toronto ON M1K 2Y5',
         'Canada'],
        'labeledLatLngs': [{'l

In [16]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [17]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The Real McCoy Burgers And Pizza,Burger Joint,43.774081,-79.230496
1,Tim Hortons,Coffee Shop,43.775992,-79.232135
2,Starbucks,Coffee Shop,43.770037,-79.221156
3,Long & McQuade,Music Store,43.775903,-79.230965
4,Morningside Park,Park,43.786546,-79.205322


In [18]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

62 venues were returned by Foursquare.


In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=800):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
toronto_venues = getNearbyVenues(names=squished['Neighborhood'],
                                   latitudes=squished['Latitude'],
                                   longitudes=squished['Longitude']
                                  )


In [21]:
print(toronto_venues.shape)
toronto_venues.head()

(3965, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Images Salon & Spa,43.802283,-79.198565,Spa
1,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
2,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.802008,-79.198080,Fast Food Restaurant
3,"Rouge, Malvern",43.806686,-79.194353,Harvey's,43.800106,-79.198258,Fast Food Restaurant
4,"Rouge, Malvern",43.806686,-79.194353,Tim Hortons,43.802000,-79.198169,Coffee Shop


In [22]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 330 uniques categories.


In [23]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Agincourt,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Alderwood, Long Branch",0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood'][:5]:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0                 Café  0.07
1          Coffee Shop  0.06
2                Hotel  0.04
3           Steakhouse  0.04
4  American Restaurant  0.04


----Agincourt----
                      venue  freq
0        Chinese Restaurant  0.11
1  Mediterranean Restaurant  0.05
2          Malay Restaurant  0.05
3            Discount Store  0.05
4           Badminton Court  0.05


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                venue  freq
0  Chinese Restaurant  0.14
1         Pizza Place  0.09
2                Park  0.09
3          Hobby Shop  0.05
4   Martial Arts Dojo  0.05


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 venue  freq
0        Grocery Store  0.20
1          Pizza Place  0.20
2       Hardware Store  0.07
3           Beer Store  0.07
4  Japanese Restaurant  0.07


----Alderwood, Long Branch----
       

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Café,Coffee Shop,American Restaurant,Steakhouse,Hotel,Gastropub,Concert Hall,Theater,Japanese Restaurant,Bar
1,Agincourt,Chinese Restaurant,Restaurant,Motorcycle Shop,Malay Restaurant,Skating Rink,Mediterranean Restaurant,Shanghai Restaurant,Breakfast Spot,Discount Store,Sandwich Place
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Chinese Restaurant,Pizza Place,Park,Hobby Shop,Bubble Tea Shop,Caribbean Restaurant,Korean Restaurant,Noodle House,Fast Food Restaurant,BBQ Joint
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Hardware Store,Fast Food Restaurant,Sandwich Place,Fried Chicken Joint,Discount Store,Japanese Restaurant,Beer Store,Pharmacy
4,"Alderwood, Long Branch",Pizza Place,Gym,Skating Rink,Athletics & Sports,Sandwich Place,Convenience Store,Bank,Pub,Gas Station,Park
5,"Bathurst Manor, Downsview North, Wilson Heights",Pizza Place,Coffee Shop,Frozen Yogurt Shop,Shopping Mall,Diner,Bank,Sandwich Place,Bridal Shop,Supermarket,Deli / Bodega
6,Bayview Village,Japanese Restaurant,Bank,Convenience Store,Grocery Store,Pharmacy,Skating Rink,Chinese Restaurant,Skate Park,Shopping Mall,Restaurant
7,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Fast Food Restaurant,Hardware Store,Pharmacy,Pizza Place,Pub,Café,Butcher,Restaurant
8,Berczy Park,Café,Hotel,Coffee Shop,Restaurant,Bakery,Cocktail Bar,Italian Restaurant,Japanese Restaurant,Park,Cheese Shop
9,"Birch Cliff, Cliffside West",Thai Restaurant,College Stadium,Diner,Skating Rink,Bank,Café,Discount Store,General Entertainment,Dive Bar,Department Store


In [28]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=42).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 4, 4, 4, 4, 4, 4, 1, 1, 1])

In [29]:
toronto_grouped_clustering.shape

(101, 329)

In [30]:
print(squished.shape)
print(kmeans.labels_.shape)

(103, 6)
(101,)


In [31]:
toronto_merged = squished.loc[squished['Neighborhood'].isin(neighborhoods_venues_sorted['Neighborhood']),:]

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

C:\Users\Tom\Anaconda37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Post Code,Borough,Neighborhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353,1,Fast Food Restaurant,Coffee Shop,Auto Workshop,Spa,African Restaurant,Filipino Restaurant,Paper / Office Supplies Store,Hobby Shop,Chinese Restaurant,Construction & Landscaping
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497,4,Breakfast Spot,Italian Restaurant,Burger Joint,Bar,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dive Bar
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711,4,Pizza Place,Coffee Shop,Fast Food Restaurant,Beer Store,Rental Car Location,Fried Chicken Joint,Sports Bar,Supermarket,Medical Center,Pharmacy
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917,4,Coffee Shop,Park,Business Service,Convenience Store,Dive Bar,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476,4,Indian Restaurant,Bakery,Pharmacy,Coffee Shop,Yoga Studio,Rental Car Location,Athletics & Sports,Bank,Board Shop,Burger Joint


In [32]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters